In [6]:
import numpy as np

### Needleman-Wunsh and Smith-Waterman

In [9]:
import numpy as np
import pandas as pd
import argparse


class Aligner:
    """ Класс для получения локального или глобального выравнивания с константными штрафами """

    def __init__(self, seq1, seq2, match=1, gap=-1, mismatch=-1, weights=False, local=False):
        """
        Инициализация аргументов
        Args:
            seq1 (str): последовательность 1
            seq2 (str): последовательность 2
            match (float): вес совпадения
            gap (float): вес пропуска
            mismatch (float): вес несовпадения
            weights (str): название матрицы весов, может быть 'pam', 'blosum', а также False - весов нет
        """

        # делаем большими и красивыми
        self.seq1 = seq1.upper()
        self.seq2 = seq2.upper()
        self.match = np.float32(match)
        self.gap = np.float32(gap)
        self.mismatch = np.float32(mismatch)

        # используется ли локальное выравнивание
        self.local = local

        # устанавливаем веса
        self.weights = self.set_weights(weights)

        # инициализируем матрицу
        self.matrix = self.init_matrix()

    def check_non_negativity(self, result):
        if self.local:
            return max(0, result)
        return result

    def init_matrix(self):
        """
        Инициализация матрицы: сначала нулями, потом заполнение для добавленных спереди пропусков
        Returns:
            np.array: инициализированная матрица
        """

        # сначала инициализируем матрицу нулями по размерам посл1 + 1, посл2 + 1; + 1 за пропуск спереди
        matrix = np.zeros([len(self.seq1) + 1, len(self.seq2) + 1])

        if self.local:
            return matrix

        # это чтобы выстаскивать веса
        seq = ['-' + self.seq1, '-' + self.seq2]

        # заполняем первую строку и первый столбец
        for i, axis in enumerate(matrix.shape):
            counter = 0

            for ax in range(axis):
                position = (0, ax) if i else (ax, 0)

                # прибавляем вес сочетания пропуска и символа
                matrix[position] = counter + self.weights.loc[('-', seq[i][ax])]

                # заполнение последовательно += self.gap
                counter += self.gap + self.weights.loc[('-', seq[i][ax])]

        return matrix

    def align(self):
        """
        Получение матрицы по алгоритму Нидлмана-Вунша
        """

        # заполняем каждую клетку
        for i in range(1, len(self.seq1) + 1):
            for j in range(1, len(self.seq2) + 1):
                self.matrix[i, j] = self.check_non_negativity(self.cell(i, j))

        # печатаем получившуюся матрицу
        self.print_matrix()

    def print_matrix(self):
        """
        Печать матрицы по алгоритму Н-В
        """

        print(pd.DataFrame(data=self.matrix, index=list(' ' + self.seq1), columns=list(' ' + self.seq2)))

    def set_weights(self, weights):
        """
        Получение матрицы весов
        Args:
            weights (str): название матрицы весов, может быть 'pam', 'blosum', а также False - весов нет
        Returns:
            pd.DataFrame: матрица весов
        """

        if weights:
            if weights.lower() == 'pam':
                filename = 'alignment/PAM250.txt'
            elif weights.lower() == 'blosum':
                filename = 'alignment/BLOSUM62.txt'
            return pd.read_csv(filename, delimiter=' ', index_col=0, header=0)

        # заполняем веса 0
        else:
            seq = set(self.seq1 + self.seq2 + '-')

            return pd.DataFrame(np.zeros([len(seq), len(seq)]), index=seq,
                                columns=seq)

    def cell(self, i, j):
        """
        Заполнение клетки матрицы по Н-В
        Args:
            i (int): индекс элемента по оси 0
            j (int): индекс по оси 1
        Returns:
            float: значение в клетке
        """

        # символы, которые выравниваем
        seq1, seq2 = self.seq1[i - 1], self.seq2[j - 1]

        # получаем троих соседей
        idx = self.get_neighbours(i, j, make_tuple=True)

        # какие веса потребуются
        weights = [(seq1, seq2), (seq1, '-'), ('-', seq2)]

        # массив, из которого будем выбирать итоговое значение (максимум)
        available = []

        # итерируемся по парам индексов соседей и ключей весов
        for index, weight in zip(idx, weights):

            # для удобства запомним индексы соседей по осям отдельно
            first_index = index[0]
            second_index = index[1]

            # получаем текущий вес
            current_weight = self.weights.loc[weight]

            # получаем значение в клетке-соседе
            current_cell = self.matrix[index]

            # если идём по диагонали
            if (i - 1 == first_index) & (j - 1 == second_index):

                # если символы совпали, то прибавляем вес совпадения
                if seq1 == seq2:
                    current_cell += self.match

                # иначе - несовпадения
                else:
                    current_cell += self.mismatch

            # если не по диагонали, то это пропуск
            else:
                current_cell += self.gap

            # доавбляем вес
            current_cell += current_weight

            available.append(current_cell)
        return max(available)

    def print_alignment(self):
        """
        Красиво печатаем выравнивание
        """
        for cell in self.alignment:
            print(f'\nAlignment starts in cell {cell}:')
            for alignment in self.alignment[cell]:
                print('\t', *alignment[::-1])

    def get_neighbours(self, i, j, make_tuple=False):
        """
        Получаем индексы трёх соседей: слева, сверху и слева сверху по диагонали
        Args:
            i (int): индекс элемента по оси 0
            j (int): индекс элемента по оси 1
            make_tuple (bool): нужно ли возвращать пары индексов в tuple (сразу для индексации) или в list
        Returns:
            list: список пар индексов
        """

        result = [[i - 1, j - 1], [i, j - 1], [i - 1, j]]

        # если надо tuple, то делаем map
        if make_tuple:
            return list(map(tuple, result))
        return result

    def select_global_alignment(self):
        """
        Получение выравненных строк
        """

        # результат выравнивания
        self.alignment = [[], []]

        # значение, из которого пойдём - правый нижний край
        current = np.array(self.matrix.shape) - 1

        # конец (не интересует сочетание двух пропусков)
        end = np.array([1, 1])

        # пока не дошли до конца
        while np.any(current >= end):

            # запоминааем текущие индексы по осям
            first_index = current[0]
            second_index = current[1]

            # получаем троих соседей
            idx = self.get_neighbours(first_index, second_index)

            # выбираем только неотрицательные, на всякий случай
            idx = np.array([i for i in idx if ((i[0] >= 0) and (i[1] >= 0))])

            # значение в текущей клетке
            value = self.matrix[tuple(current)]

            # по каждому соседу
            for index in idx:

                # символы, которые сейчас сравниваем
                seq1 = self.seq1[current[0] - 1]
                seq2 = self.seq2[current[1] - 1]

                # значение в клетке-соседе
                backtrace = self.matrix[tuple(index)]

                # если идём по диагонали
                if np.array_equal(index + 1, current):

                    # берём вес от двух символов
                    weight = self.weights.loc[(seq1, seq2)]

                    # если символы совпадают и мы пришли из клетки по диагонали или символы по диагонали не совпали
                    if ((value - backtrace == self.match + weight) and (seq1 == seq2)) or \
                            (value - backtrace == self.mismatch + weight):

                        # добавляем в результат оба символа
                        self.alignment[0].append(seq1)
                        self.alignment[1].append(seq2)

                        # переходим в клетку-соседа
                        current = index
                        break

                # если идём наверх
                elif current[0] - index[0]:

                    # берём вес для символа первой последовательности и пропуска
                    weight = self.weights.loc[(seq1, '-')]
                    if value - backtrace == self.gap + weight:
                        self.alignment[0].append(seq1)
                        self.alignment[1].append('-')
                        current = index
                        break

                # если идём налево, то пропуск и символ второй последовательности
                else:
                    weight = self.weights.loc[('-', seq2)]
                    if value - backtrace == self.gap + weight:
                        self.alignment[0].append('-')
                        self.alignment[1].append(seq2)
                        current = index
                        break
        self.alignment = {tuple(np.array(self.matrix.shape) - 1): self.alignment}

    def select_local_alignment(self):
        matrix_max = self.matrix.max()
        indices = np.where(self.matrix == matrix_max)
        indices = [(x, y) for x, y in zip(*indices)]

        self.alignment = {}

        for start in indices:
            # значение в текущей клетке
            value = self.matrix[tuple(start)]
            current = start

            alignment = [[], []]

            while value != 0:
                # запоминааем текущие индексы по осям
                first_index = current[0]
                second_index = current[1]

                # получаем троих соседей
                idx = self.get_neighbours(first_index, second_index)

                # выбираем только неотрицательные, на всякий случай
                idx = np.array([i for i in idx if ((i[0] >= 0) and (i[1] >= 0))])

                # значение в текущей клетке
                value = self.matrix[tuple(current)]

                # по каждому соседу
                for index in idx:

                    # символы, которые сейчас сравниваем
                    seq1 = self.seq1[current[0] - 1]
                    seq2 = self.seq2[current[1] - 1]

                    # значение в клетке-соседе
                    backtrace = self.matrix[tuple(index)]

                    # если идём по диагонали
                    if np.array_equal(index + 1, current):

                        # берём вес от двух символов
                        weight = self.weights.loc[(seq1, seq2)]

                        # если символы совпадают и мы пришли из клетки по диагонали или символы по диагонали не совпали
                        if ((value - backtrace == self.match + weight) and (seq1 == seq2)) or \
                                (value - backtrace == self.mismatch + weight):
                            # добавляем в результат оба символа
                            alignment[0].append(seq1)
                            alignment[1].append(seq2)

                            # переходим в клетку-соседа
                            current = index
                            break

                    # если идём наверх
                    elif current[0] - index[0]:

                        # берём вес для символа первой последовательности и пропуска
                        weight = self.weights.loc[(seq1, '-')]
                        if value - backtrace == self.gap + weight:
                            alignment[0].append(seq1)
                            alignment[1].append('-')
                            current = index
                            break

                    # если идём налево, то пропуск и символ второй последовательности
                    else:
                        weight = self.weights.loc[('-', seq2)]
                        if value - backtrace == self.gap + weight:
                            alignment[0].append('-')
                            alignment[1].append(seq2)
                            current = index
                            break

            self.alignment[tuple(start)] = alignment

In [10]:
aligner = Aligner(seq1='GTAGGCTTAAGGTTA',seq2='TAGATA', match=1, gap=-1, mismatch=-1)
aligner.align()
aligner.select_global_alignment()
aligner.print_alignment()

            T     A    G    A    T    A
    0.0  -1.0  -2.0 -3.0 -4.0 -5.0 -6.0
G  -1.0  -1.0  -2.0 -1.0 -2.0 -3.0 -4.0
T  -2.0   0.0  -1.0 -2.0 -2.0 -1.0 -2.0
A  -3.0  -1.0   1.0  0.0 -1.0 -2.0  0.0
G  -4.0  -2.0   0.0  2.0  1.0  0.0 -1.0
G  -5.0  -3.0  -1.0  1.0  1.0  0.0 -1.0
C  -6.0  -4.0  -2.0  0.0  0.0  0.0 -1.0
T  -7.0  -5.0  -3.0 -1.0 -1.0  1.0  0.0
T  -8.0  -6.0  -4.0 -2.0 -2.0  0.0  0.0
A  -9.0  -7.0  -5.0 -3.0 -1.0 -1.0  1.0
A -10.0  -8.0  -6.0 -4.0 -2.0 -2.0  0.0
G -11.0  -9.0  -7.0 -5.0 -3.0 -3.0 -1.0
G -12.0 -10.0  -8.0 -6.0 -4.0 -4.0 -2.0
T -13.0 -11.0  -9.0 -7.0 -5.0 -3.0 -3.0
T -14.0 -12.0 -10.0 -8.0 -6.0 -4.0 -4.0
A -15.0 -13.0 -11.0 -9.0 -7.0 -5.0 -3.0

Alignment starts in cell (15, 6):
	 G T A G G C T T A A G G T T A
	 - T A - G - - - - A - - - T A


In [11]:
aligner = Aligner(seq1='GTAGGCTTAAGGTTA',seq2='TAGATA', match=1, gap=-1, mismatch=-1, local=True)
aligner.align()
aligner.select_local_alignment()
aligner.print_alignment()

          T    A    G    A    T    A
   0.0  0.0  0.0  0.0  0.0  0.0  0.0
G  0.0  0.0  0.0  1.0  0.0  0.0  0.0
T  0.0  1.0  0.0  0.0  0.0  1.0  0.0
A  0.0  0.0  2.0  1.0  1.0  0.0  2.0
G  0.0  0.0  1.0  3.0  2.0  1.0  1.0
G  0.0  0.0  0.0  2.0  2.0  1.0  0.0
C  0.0  0.0  0.0  1.0  1.0  1.0  0.0
T  0.0  1.0  0.0  0.0  0.0  2.0  1.0
T  0.0  1.0  0.0  0.0  0.0  1.0  1.0
A  0.0  0.0  2.0  1.0  1.0  0.0  2.0
A  0.0  0.0  1.0  1.0  2.0  1.0  1.0
G  0.0  0.0  0.0  2.0  1.0  1.0  0.0
G  0.0  0.0  0.0  1.0  1.0  0.0  0.0
T  0.0  1.0  0.0  0.0  0.0  2.0  1.0
T  0.0  1.0  0.0  0.0  0.0  1.0  1.0
A  0.0  0.0  2.0  1.0  1.0  0.0  2.0

Alignment starts in cell (4, 3):
	 T A G
	 T A G


In [7]:
class AffineAligner:
    """ Класс для получения глобального выравнивания с аффинными штрафами """

    def __init__(self, seq1, seq2, match=1, gap_start=-1, gap_continued=-0.5,  mismatch=-1):
        """
        Инициализация аргументов
        Args:
            seq1 (str): последовательность 1
            seq2 (str): последовательность 2
            match (float): вес совпадения
            gap_start (float): вес начала пропуска
            gap_continued (float): вес продолжения пропуска
            mismatch (float): вес несовпадения
        """

        # делаем большими и красивыми
        self.seq1 = seq1.upper()
        self.seq2 = seq2.upper()
        self.match = np.float32(match)
        self.gap_start = np.float32(gap_start)
        self.gap_continued = np.float32(gap_continued)
        self.mismatch = np.float32(mismatch)

        # инициализируем матрицы
        self.middle = self.init_middle()
        self.upper = self.init_lower_upper('upper')
        self.lower = self.init_lower_upper('lower')

    def init_middle(self):
        """
        Инициализируем матрицу диагоналей (middle)
        Returns:
            np.array: инициализированная middle матрица
        """
        # сначала инициализируем матрицу нулями по размерам посл1 + 1, посл2 + 1; + 1 за пропуск спереди
        matrix = np.zeros([len(self.seq1) + 1, len(self.seq2) + 1])
        matrix[1:, 0] = -np.inf
        matrix[0, 1:] = -np.inf
        return matrix

    def init_lower_upper(self, which):
        """
        Инициализируем матрицы с гэпами (lower, upper)
        Args:
            which (str): тип матрицы (lower, upper)
        Returns:
            np.array: инициализированная матрица
        """

        # сначала всё нули
        matrix = np.zeros([len(self.seq1) + 1, len(self.seq2) + 1])

        # в зависимости от того, какую матрицу заполняем, нужные оси заполняем -inf и штрафами
        if which == 'lower':
            for i in range(matrix.shape[1]):
                matrix[0, i] = self.init_cell(i)
            matrix[1:, 0] = -np.inf

        else:
            for i in range(matrix.shape[0]):
                matrix[i, 0] = self.init_cell(i)
            matrix[0, 1:] = -np.inf
        return matrix

    def get_indices(self, i, j):
        """
        Получение индексов соседей
        Args:
            i (int): индекс текцщей клетки по оси Ox
            j (int): индекс текцщей клетки по оси Oy
        Returns:
            tuple: три tuple - по одному на матрицу
        """
        return (i - 1, j - 1), (i - 1, j), (i, j - 1)

    def init_cell(self, cnt):
        """
        Инициализируем клетку, где должен быть штраф
        Args:
            cnt (int): множитель для функции
        Returns:
            float: штраф в клетке
        """
        return self.gap_start + cnt * self.gap_continued

    def seq_match(self, i, j):
        """
        Проверка на совпадение
        Args:
            i (int): индекс текцщей клетки по оси Ox
            j (int): индекс текцщей клетки по оси Oy
        Returns:
            int: результат
        """
        return 1 if self.seq1[i - 1] == self.seq2[j - 1] else -1

    def align(self):
        """
        Заполнение матриц
        """
        for i in range(1, len(self.seq1) + 1):
            for j in range(1, len(self.seq2) + 1):
                middle, upper, lower = self.get_indices(i, j)
                match = self.seq_match(i, j)

                self.upper[i, j] = max(self.middle[upper] + self.gap_start + self.gap_continued,
                                       self.upper[upper] + self.gap_continued)
                self.lower[i, j] = max(self.middle[lower] + self.gap_start + self.gap_continued,
                                       self.lower[lower] + self.gap_continued)
                self.middle[i, j] = max(self.middle[middle] + match,
                                        self.upper[middle] + match,
                                        self.lower[middle] + match)
        print('\nUPPER:')
        print(self.upper)

        print('\nMIDDLE:')
        print(self.middle)

        print('\nLOWER:')
        print(self.lower)

    def check_index(self, index):
        """
        Проверка индексов на отрицательность
        Args:
            index (tuple): индекс (х, у) для проверки
        Returns:
            bool: результат проверки
        """
        index = np.array(index)
        return np.any(index >= np.array([0, 0]))

    def select_alignment(self):
        """
        Выбор выравнивания
        """
        # значение, из которого пойдём - правый нижний край
        current = tuple(np.array(self.middle.shape) - 1)

        # результат выравнивания
        self.alignment = [[], []]

        # конец (не интересует сочетание двух пропусков)
        end = np.array([1, 1])

        # текущие точки в трёх матрицах
        variants = np.array([self.middle[current], self.upper[current], self.lower[current]])

        # сначала выбираем, из какой матрицы стартовать (если есть одинаковые значения, то пробуем обе)
        value = np.where(variants == variants.max())[0]

        while np.any(current >= end):

            # запоминааем текущие индексы по осям
            first_index = current[0]
            second_index = current[1]

            # если ещё есть символы в какой либо строке, то продолжаем
            # иначе просто дописываем выравнивание
            if first_index:
                seq1 = self.seq1[first_index - 1]
            else:
                self.alignment[0].append('-')
                self.alignment[1].append(self.seq2[second_index - 1])
                current = np.array([first_index, second_index - 1])
                continue

            if second_index:
                seq2 = self.seq2[second_index - 1]
            else:
                self.alignment[0].append(seq1)
                self.alignment[1].append('-')
                current = np.array([first_index - 1, second_index])
                continue

            # получаем индексы для трёх матриц
            middle, upper, lower = self.get_indices(first_index, second_index)

            # получаем результаты проверки на неотрицвательность индексов
            nonnegative = [x for x in map(self.check_index, [middle, upper, lower])]

            # проверяем, одинковые ли буквы сейчас
            match = self.seq_match(first_index, second_index)

            # по всем матрицам, где нашли подходящие значния
            for matrix in value:

                # если сейчас можно идти по этой матрице
                if nonnegative[matrix]:

                    # в зависимости от типа матрицы
                    if matrix == 0:
                        # если middle (0)

                        variants = np.array([self.middle[middle] + match,
                                             self.upper[middle] + match,
                                             self.lower[middle] + match])
                        middle_max = variants.max()

                        # если пришли из неё, то добавляем нужные символы в выравнивание
                        if self.middle[current] == middle_max:
                            value = np.where(variants == middle_max)[0]
                            self.alignment[0].append(seq1)
                            self.alignment[1].append(seq2)
                            current = middle
                            break

                    # если из upper (1)
                    elif matrix == 1:
                        variants = np.array([self.middle[upper] + self.gap_start + self.gap_continued,
                                             self.upper[upper] + self.gap_continued])
                        upper_max = variants.max()
                        if self.upper[current] == upper_max:
                            value = np.where(variants == upper_max)[0]
                            self.alignment[0].append(seq1)
                            self.alignment[1].append('-')
                            current = upper
                            break

                    # если из lower (2)
                    elif matrix == 2:
                        variants = np.array([self.middle[lower] + self.gap_start + self.gap_continued,
                                             self.lower[lower] + self.gap_continued])
                        lower_max = variants.max()

                        if self.lower[current] == lower_max:
                            value = np.where(variants == lower_max)[0]
                            self.alignment[0].append('-')
                            self.alignment[1].append(seq2)
                            current = lower
                            break

    def print_alignment(self):
        """
        Печать выравнивания
        """
        for seq in self.alignment:
            print('\t', *seq[::-1])

In [17]:
aligner_af = AffineAligner(seq1='GTAGGCTTAAGGTTA',seq2='TAGATA', match=1, gap_start=-1, mismatch=-0.5, gap_continued=-1)
aligner_af.align()
aligner_af.select_alignment()
aligner_af.print_alignment()


UPPER:
[[ -1. -inf -inf -inf -inf -inf -inf]
 [ -2. -inf -inf -inf -inf -inf -inf]
 [ -3.  -3.  -5.  -4.  -7.  -8.  -9.]
 [ -4.  -3.  -4.  -5.  -5.  -5.  -8.]
 [ -5.  -4.  -2.  -5.  -5.  -6.  -4.]
 [ -6.  -5.  -3.  -1.  -5.  -6.  -5.]
 [ -7.  -6.  -4.  -2.  -2.  -4.  -5.]
 [ -8.  -7.  -5.  -3.  -3.  -3.  -5.]
 [ -9.  -8.  -6.  -4.  -4.  -3.  -4.]
 [-10.  -9.  -7.  -5.  -5.  -4.  -4.]
 [-11. -10.  -8.  -6.  -5.  -5.  -3.]
 [-12. -11.  -9.  -7.  -6.  -6.  -4.]
 [-13. -12. -10.  -8.  -7.  -7.  -5.]
 [-14. -13. -11.  -9.  -8.  -8.  -6.]
 [-15. -14. -12. -10.  -9.  -8.  -7.]
 [-16. -15. -13. -11. -10.  -9.  -8.]]

MIDDLE:
[[  0. -inf -inf -inf -inf -inf -inf]
 [-inf  -1.  -3.  -2.  -5.  -6.  -7.]
 [-inf  -1.  -2.  -4.  -3.  -3.  -6.]
 [-inf  -4.   0.  -3.  -3.  -4.  -2.]
 [-inf  -5.  -4.   1.  -3.  -4.  -5.]
 [-inf  -6.  -5.  -1.   0.  -2.  -3.]
 [-inf  -7.  -6.  -4.  -2.  -1.  -3.]
 [-inf  -6.  -7.  -5.  -3.  -1.  -2.]
 [-inf  -7.  -7.  -6.  -4.  -2.  -2.]
 [-inf -10.  -6.  -7.  -3.  -5. 

### Fitting

In [ ]:
def LRS(str1, str2):
    """
    Finding longest repeated substring of given string. Repetitions may overlap.
    In this function dynamic programing is used.
    Args:
        str: string to find LRS
    Return:
        str: found LRS
        2D-array: matrix which was computed to find LRS 
    """
    n_1 = len(str1)
    n_2 = len(str2_
    # Initialiazing matrix
    matrix = [[0 for x in range(n_1)]  
                for y in range(n_2)] 
    res = ""
    res_length = 0
    index = 0
    # Calculating matrix
    for i in range(1, n_1): 
        for j in range(i + 1, n_2): 
            '''#if (string[i - 1] == string[j - 1] and
                #matrix[i - 1][j - 1] < (j - i)):'''
            # If last characters are same:
            if (str1[i-1] == str2[j-1]):
                # Continue diag path
                matrix[i][j] = matrix[i-1][j-1] + 1
                # If-constuction to find maximum in matrix 
                if (matrix[i][j] > res_length): 
                    res_length = matrix[i][j] 
                    index = (i, j)
            # Terminating diag path
            else: 
                matrix[i][j] = 0
    # Backtracking
    a, b = index
    while a!=0 and b!=0:
              if matrix[a][b]
    return res, matrix 